In [1]:
# pip install selenium beautifulsoup4 pandas mysql-connector-python

In [2]:
# pip install selenium

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import time
import pandas as pd
import mysql.connector

In [5]:
chrome_path = "./chromedriver.exe"
service = Service(executable_path=chrome_path)
driver = webdriver.Chrome(service=service)

driver.get("https://www.google.com")
print("크롬 실행 성공!")
driver.quit()

크롬 실행 성공!


In [8]:
# 크롬 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창 없이 실행
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# 드라이버 경로 설정 (수정 필요)
service = Service('./chromedriver.exe')  # 여기 경로로 바꿔야됨.
driver = webdriver.Chrome(service=service, options=chrome_options)

# 검색어
keyword = '연두색 번호판'
# 네이버 뉴스 검색 URL
url = f"https://search.naver.com/search.naver?where=news&query={keyword}"
driver.get(url)
time.sleep(3)  # 로딩 대기

# 페이지 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 뉴스 기사 박스 선택
articles = soup.select("div.news_area")

if not articles:
    print("❗ 뉴스 기사를 찾을 수 없습니다.")
else:
    for article in articles:
        title_tag = article.select_one("a.news_tit")
        press_tag = article.select_one("a.info.press")
        date_tag = article.select_one("span.info")

        title = title_tag["title"].strip() if title_tag else "제목 없음"
        link = title_tag["href"] if title_tag else "링크 없음"
        press = press_tag.text.strip() if press_tag else "언론사 없음"
        date = date_tag.text.strip() if date_tag else "날짜 없음"

        print(f"제목: {title}")
        print(f"링크: {link}")
        print(f"언론사: {press}")
        print(f"날짜: {date}")
        print("-" * 50)

driver.quit()


제목: 연두색 번호판의 기습…S클래스 독주 끝? [데이터로 보는 세상]
링크: https://www.mk.co.kr/article/11276278
언론사: 매경이코노미
날짜: 3일 전
--------------------------------------------------
제목: '연두색 번호판' 기피 현상 감소…고급 수입차 판매 '꿈틀'
링크: https://www.newsis.com/view/NISX20250324_0003110599
언론사: 뉴시스언론사 선정
날짜: 2주 전
--------------------------------------------------
제목: 번호판도 싫고, 살 차도 없고… 고가 수입차 '내리막'
링크: https://www.dailian.co.kr/news/view/1477573/?sc=Naver
언론사: 데일리안언론사 선정
날짜: 1주 전
--------------------------------------------------
제목: "정체된 S클래스"···벤츠, 더 위 노린다
링크: https://www.sisajournal-e.com/news/articleView.html?idxno=410659
언론사: 시사저널이코노미
날짜: 7일 전
--------------------------------------------------
제목: 연두색 번호판 기피 현상 감소? 올해 들어 고급 수입차 판매 늘어
링크: http://www.wolyo.co.kr/news/articleView.html?idxno=257386
언론사: 월요신문
날짜: 2주 전
--------------------------------------------------
제목: '맞수' 벤츠 S클래스·BMW 7시리즈…'영원한 왕좌'는 없다?
링크: https://www.gpkorea.com/news/articleView.html?idxno=126703
언론사: 지피코리아
날짜: 1주 전
----------------------------------------

In [6]:
import csv
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# 🧩 기사 본문 추출 함수
def extract_article_content(driver, link):
    try:
        driver.get(link)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")

        if "mk.co.kr" in link:
            content = soup.select_one("div.art_txt")
        elif "newsis.com" in link:
            content = soup.select_one("div#textBody") or soup.select_one("div.view_con_tx")
        elif "dailian.co.kr" in link:
            content = soup.select_one("div.view-content") or soup.select_one("div#viewContent")
        elif any(domain in link for domain in ["sisajournal-e.com", "wolyo.co.kr", "gpkorea.com", "hansbiz.co.kr"]):
            content = soup.select_one("div#article-view-content-div")
        elif "chosun.com" in link:
            content = soup.select_one("div#news_article")
        elif "segye.com" in link:
            content = soup.select_one("div#article_txt")
        else:
            content = None

        return content.get_text(separator=" ", strip=True) if content else "본문 추출 실패"

    except Exception as e:
        print(f"[에러] {link} 본문 추출 실패: {e}")
        return "본문 추출 오류"

# 🧰 크롬 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# 드라이버 경로 수정 필요
service = Service('./chromedriver.exe')  # 👉 여기를 본인 PC에 맞게 수정
driver = webdriver.Chrome(service=service, options=chrome_options)

# 🔍 검색어 설정
keyword = "연두색 번호판"
search_url = f"https://search.naver.com/search.naver?where=news&query={keyword}"
driver.get(search_url)
time.sleep(3)

# 페이지 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")
articles = soup.select("div.news_area")

results = []

# 뉴스 제목 & 링크 수집 및 본문 추출
for article in articles:
    title_tag = article.select_one("a.news_tit")
    if not title_tag:
        continue

    title = title_tag["title"].strip()
    link = title_tag["href"]

    print(f"📄 기사 추출 중: {title}")
    content = extract_article_content(driver, link)

    results.append({
        "제목": title,
        "본문": content,
        "링크": link
    })

# 📝 CSV로 저장
csv_filename = "naver_news.csv"
with open(csv_filename, mode="w", encoding="utf-8-sig", newline="") as file:
    writer = csv.DictWriter(file, fieldnames=["제목", "본문", "링크"])
    writer.writeheader()
    writer.writerows(results)

print(f"\n✅ {len(results)}개의 기사를 '{csv_filename}'에 저장 완료!")

driver.quit()


📄 기사 추출 중: 연두색 번호판의 기습…S클래스 독주 끝? [데이터로 보는 세상]
📄 기사 추출 중: '연두색 번호판' 기피 현상 감소…고급 수입차 판매 '꿈틀'
📄 기사 추출 중: 번호판도 싫고, 살 차도 없고… 고가 수입차 '내리막'
📄 기사 추출 중: "정체된 S클래스"···벤츠, 더 위 노린다
📄 기사 추출 중: 연두색 번호판 기피 현상 감소? 올해 들어 고급 수입차 판매 늘어
📄 기사 추출 중: '맞수' 벤츠 S클래스·BMW 7시리즈…'영원한 왕좌'는 없다?
📄 기사 추출 중: 연두색 번호판 탓?… 고가 수입차, 영리치 대신 50대가 사
📄 기사 추출 중: '연두색 번호판'에도 본국보다 판매 많아…4억 '테메라리오' 상륙
📄 기사 추출 중: ‘연두색 아니네’…공수처장 관용차 번호판 바꿀 수 있던 이유는
📄 기사 추출 중: “연두색 번호판 창피해서?”···수입차 시세 8년만에 ‘뚝뚝’

✅ 10개의 기사를 'naver_news.csv'에 저장 완료!


In [7]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options

def extract_article_content(driver, url):
    try:
        driver.get(url)
        time.sleep(2)  # 페이지 로딩 대기
        soup = BeautifulSoup(driver.page_source, 'html.parser')

        # 언론사별 본문 추출 로직
        if 'mk.co.kr' in url:
            content = soup.select_one('div#article_body')
        elif 'newsis.com' in url:
            content = soup.select_one('div#textBody')
        elif 'dailian.co.kr' in url:
            content = soup.select_one('div#articleContent')
        elif 'sisajournal-e.com' in url or 'wolyo.co.kr' in url or 'gpkorea.com' in url or 'hansbiz.co.kr' in url:
            content = soup.select_one('div#article-view-content-div')
        elif 'chosun.com' in url:
            content = soup.select_one('div#news_body_id')
        elif 'segye.com' in url:
            content = soup.select_one('div.article_txt')
        else:
            print(f"❌ [{url}] 해당 언론사에 대한 본문 추출 로직이 없습니다.")
            return None

        if content:
            return content.get_text(separator=' ', strip=True)
        else:
            print(f"❌ [{url}] 본문을 찾을 수 없습니다.")
            return None

    except Exception as e:
        print(f"❌ [{url}] 본문 추출 중 에러 발생: {e}")
        return None

# 크롬 드라이버 설정
chrome_options = Options()
chrome_options.add_argument("--headless")  # 브라우저 창을 띄우지 않음
service = Service('./chromedriver.exe')  # chromedriver 경로 설정
driver = webdriver.Chrome(service=service, options=chrome_options)

# 테스트할 기사 URL 리스트
article_urls = [
    'https://www.newsis.com/view/NISX20250324_0003110599',
    # 다른 기사 URL 추가 가능
]

# 각 기사에 대해 본문 추출 테스트
for url in article_urls:
    print(f"🔗 기사 URL: {url}")
    content = extract_article_content(driver, url)
    if content:
        print(f"📝 본문 내용: {content[:100]}...")  # 본문 내용의 앞부분만 출력
    print("-" * 80)

driver.quit()


🔗 기사 URL: https://www.newsis.com/view/NISX20250324_0003110599
📝 본문 내용: [수원=뉴시스] 김종택 기자 = 지난해 1월8일 경기도 수원시 권선구 수원도시공사 번호판제작소에서 직원이 고가 법인차량 사적 사용과 탈세를 막기 위해 도입된 연두색 차량 번호판을 ...
--------------------------------------------------------------------------------


In [8]:
content # 본문 크롤링을 잘못하고있음.

'[수원=뉴시스] 김종택 기자 = 지난해 1월8일 경기도 수원시 권선구 수원도시공사 번호판제작소에서 직원이 고가 법인차량 사적 사용과 탈세를 막기 위해 도입된 연두색 차량 번호판을 정리하고 있다. 2024.01.08. jtk@newsis.com'

In [9]:
import requests
from bs4 import BeautifulSoup
# keyword = "법인차량"
url = ("https://www.newsis.com/view/NISX20250324_0003110599")

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.select_one('#content > div.articleView > div.view > div.viewer > article')
    print(title)
else : 
    print(response.status_code)

<article>
<div class="summury">
<h2>'연두색 번호판' 수입 법인차 증가세<br/></h2><h2>2024년 시행 직후 연간 판매량 감소<br/></h2><h2>1~2월 BMW·벤츠 판매율 반등 조짐<br/></h2><h2>람보르기니 630%, 벤틀리 47% 증가</h2>
</div>
<div id="view_ad"></div>
<div align="justify" class="thumCont"><div id="textBody"><div class="article_photo thum center" id="imgartitable_NISI20240108_0020186673"><tbody><tr><td class="img"><img alt="[수원=뉴시스] 김종택 기자 = 지난해 1월8일 경기도 수원시 권선구 수원도시공사 번호판제작소에서 직원이 고가 법인차량 사적 사용과 탈세를 막기 위해 도입된 연두색 차량 번호판을 정리하고 있다. 2024.01.08. jtk@newsis.com" arti_id="NISI20240108_0020186673" check_caption="Y" mode_we="edit" src="https://img1.newsis.com/2024/01/08/NISI20240108_0020186673_web.jpg?rnd=20240108135648" style="float:; margin:0 auto;display:block;"/></td></tr><tr><p class="desc" id="caption_NISI20240108_0020186673"><p class="photojournal">[수원=뉴시스] 김종택 기자 = 지난해 1월8일 경기도 수원시 권선구 수원도시공사 번호판제작소에서 직원이 고가 법인차량 사적 사용과 탈세를 막기 위해 도입된 연두색 차량 번호판을 정리하고 있다. 2024.01.08. <a class="__cf_email__" data-cfemail="93f9e7f8d3fdf6e4e0fae0bdf0fcfe" h

In [10]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.newsis.com/view/NISX20250324_0003110599'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(response.content, 'html.parser')

# 1. 전체 기사 영역
article_body = soup.select_one('#content > div.articleView > div.view > div.viewer > article')

if article_body:
    # 2. 불필요한 태그 제거 (사진, 캡션 등)
    for tag in article_body.select('img, p.desc, p.photojournal, h2, table, script, style'):
        tag.decompose()

    # 3. 텍스트만 추출
    clean_text = article_body.get_text(separator="\n", strip=True)
    print(clean_text)
else:
    print("본문 영역을 찾을 수 없습니다.")

[서울=뉴시스]박현준 기자 = 지난해부터 시행한 '연두색 번호판' 제도로 고급 수입 법인차 판매량이 급감했지만, 올해 다시 반등 조짐을 보이고 있다.
24일 한국수입자동차협회(KAIDA)에 따르면 지난달까지 법인으로 등록한 1억원 이상 수입차는 5501대로 파악됐다. 지난해 같은 기간(4394대) 대비 25%가량 증가한 수치다.
지난해 초부터 정부는 법인 차량의 사적 사용을 막기 위해 취득가액 8000만원 이상 차량엔 의무적으로 연두색 번호판을 붙이는 정책을 시행했다.
이에 고급 수입 법인차부터 고위 '럭셔리 카' 브랜드에 이르는 차량 판매가 감소하는 현상이 나타났다.
지난 2023년 1억원 이상 수입 법인차의 연간 판매량은 5만1083대였다. 하지만 정책이 시행된 2024년 한 해 동안 3만5320대가 팔리며 연간 판매량이 30% 넘게 주저앉았다.
이 중에서도 소위 '럭셔리 카'로 분류되는 수입 브랜드의 감소 폭이 컸다. 2023년 1~2월과 지난해 같은 기간의 1억원 이상 법인 차량의 판매량을 비교했을 때 벤틀리(-83.2%), 람보르기니(-73.7%), 롤스로이스(-43.3%) 등에서 큰 감소세가 나타났다.
하지만 올해 1~2월 1억원 이상 BMW 법인 차량 판매량은 전년보다 31.8% 증가한 2296대, 메르세데스-벤츠도 15.6% 증가한 1546대로 판매율이 반등할 조짐이다.
특히 람보르기니의 경우 지난해 같은 기간 대비 630% 증가한 73대를 판매하는 등 고가 법인차 시장을 이끄는 것으로 나타났다. 벤틀리 역시 47.1% 증가한 25대를 기록했고, 롤스로이스는 전년과 비슷한 수준인 14대로 집계됐다.
업계 관계자는 "연두색 번호판을 도입한 지 1년이 넘으며 기피 현상이 점점 줄어들고 있다"며 "고급 법인차량 판매가 급감한 지난해보다 올해 판매량이 더 늘어날 것으로 예상된다"고 말했다.
◎공감언론 뉴시스
[email protected]


In [7]:
import requests
from bs4 import BeautifulSoup
# keyword = "법인차량"
url = ("https://www.sisajournal-e.com/news/articleView.html?idxno=406663")

response = requests.get(url)

if response.status_code == 200:
    html = response.text
    soup = BeautifulSoup(html, 'html.parser')
    title = soup.select_one('#article-view-content-div')
    print(title)
else : 
    print(response.status_code)

<article class="article-veiw-body view-page" id="article-view-content-div" itemprop="articleBody">
<p style="text-align: justify;">[시사저널e=박성수 기자] 올해 애스턴마틴, 마세라티 등 슈퍼카 브랜드들이 한국 시장 판매 확대를 위한 시동을 걸고 있다.</p>
<p style="text-align: justify;">그동안 두 브랜드의 경우 국내 시장에선 다소 소극적인 움직임을 보였으나 올해에는 한국 법인 출범 및 전시장 신설, 신차 출시 등을 통해 존재감을 드러내고 있다.</p>
<p style="text-align: justify;">23일 업계에 따르면 애스턴마틴은 올해 신규 공식 수입사로 브리타니아 오토를 선정하고 서울과 수원에서 전시장을 열고 국내 판매 확대에 나섰다.</p>
<p style="text-align: justify;">그동안 애스턴마틴은 국내에서 마니아층을 중심으로 꾸준한 인기를 끌었으며 올해에는 한국 진출 10주년을 맞아 판매 네트워크를 강화해 판매량을 더 높이겠다는 전략이다.</p>
<div style="text-align:center">
<figure class="photo-layout image photo_210661 float-center" data-idxno="210661" data-type="photo" style="display:inline-block; max-width:960px"><img alt="뱅퀴시. / 사진=애스턴마틴" height="640" src="https://cdn.sisajournal-e.com/news/photo/202410/406663_210661_125.jpg" width="960">
<figcaption>뱅퀴시. / 사진=애스턴마틴</figcaption>
</img></figure>
</div>
<p>동시에 플래그십 스포츠카 ‘뱅퀴시’ 신형도 국내 출시한다. 뱅퀴시는 5.2ℓ 트윈 터보 V12 

In [21]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import time

driver = webdriver.Chrome()

url = "https://www.dt.co.kr/contents.html?article_no=2024111002109932091005&ref=naver"
driver.get(url)

time.sleep(3)  # 페이지 로딩 대기

# 예: 본문 태그를 자동으로 찾기
try:
    element = driver.find_element(By.CSS_SELECTOR, "#article-view-content-div")
    #v-left-scroll-in > div.article_con > div > div.article_view
    print(element.text)
except:
    print("본문 못 찾음")

driver.quit()


본문 못 찾음


In [18]:
print("hi")

hi


In [22]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.heumtax.com/contents/posts/corporate-car-green-plate'
response = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
soup = BeautifulSoup(response.content, 'html.parser')

# 1. 전체 기사 영역
article_body = soup.select_one('body > div > div > div > div > article > div > div > div')

if article_body:
    # 2. 불필요한 태그 제거 (사진, 캡션 등)
    for tag in article_body.select('img, p.desc, p.photojournal, h2, table, script, style'):
        tag.decompose()

    # 3. 텍스트만 추출
    clean_text = article_body.get_text(separator="\n", strip=True)
    print(clean_text)
else:
    print("본문 영역을 찾을 수 없습니다.")

도로 위의
연두색 번호판, 법인차
최근 정부는
법인 등록 차량의 사적 사용을 방지
하기 위해
연두색 번호판(연두색 바탕+흰색 문자) 제도
를 도입했습니다. 2024년 1월부터
법인으로 구매하고자 하는 차량 가격이 8,000만 원 이상인 법인 소유의 차량에는 ‘녹색 번호판’이 의무적으로 부착
되며, 이를 통해 법인 차량의 사적 사용을 보다 체계적으로 관리할 수 있게 되었습니다.
이제는 법인 차량이 연두색 번호판으로 구분되면서,
업무용 차량을 개인적으로 사용하는 것이 더 이상 쉽지 않게
되었습니다. 이에 따라 국세청의
업무용 차량 관련 세무조사도 더욱 강화될 전망
이며, 사적 사용이 적발될 경우 세금 추징 및 가산세 부과 위험이 커질 수 있습니다.
그렇다면, 업무용 승용차를 올바르게 사용하고
세금 추징을 방지하려면 어떻게 해야 할까요?
아래에서
세무조사 적발 사례와 예방 방법
을 알아보겠습니다.
by
전효진 세무사(세무법인 혜움 컨설팅팀)
업무용 승용차는 반드시 회사의 사업 목적에 맞게 사용해야 합니다. 하지만 일부 기업에서는 이를 개인 용도로 활용하면서 비용을 법인에서 부담하는 사례가 적발되고 있습니다.
💡
국세청이 문제 삼는 대표적인 사례
업무용 차량을 실제로는 사적으로 사용
하면서 차량 비용을 전액 법인 비용으로 처리
운행기록부를 작성하지 않거나, 허위로 작성
하여 비용을 부당하게 공제
법인의
대표이사·임원·주주 등이 개인 차량처럼 사용
하면서 비용을 법인에 전가
비업무용 차량(고급 수입차 등)을 회사 명의로 구입
하여 실질적으로는 대표자의 사적 용도로 활용
이러한 사례들이 적발되면
법인세, 소득세, 부가가치세 등이 추가 추징될 수 있으며, 경우에 따라 가산세까지 부과
될 수 있습니다.
1️⃣
법인 대표의 가족이 회사 차량을 사적으로 사용한 사례
A법인은 업무용 차량으로 등록된 고급 수입차를 구매한 후,
대표이사의 배우자와 자녀가 주말·휴일에 개인 용도로 사용
했습니다. 그러나 운행기록부를 작성하지 않았고, 차량 유지비 전액을 법인 비용으로 처리했습니다.
국

In [12]:
import requests
from bs4 import BeautifulSoup

def get_unique_selector(element):
    path = []
    while element.parent:
        siblings = element.parent.find_all(element.name, recursive=False)
        if len(siblings) > 1:
            index = siblings.index(element) + 1
            path.append(f'{element.name}:nth-of-type({index})')
        else:
            path.append(element.name)
        element = element.parent
    return ' > '.join(reversed(path))

url = 'https://www.sisajournal-e.com/news/articleView.html?idxno=406663'

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'lxml')

target_text_snippet = '올해'

target_element = soup.find(lambda tag: tag.name and tag.get_text() and target_text_snippet in tag.get_text())

if target_element:
    selector = get_unique_selector(target_element)
    print("자동으로 찾은 CSS Selector:", selector)
    print("본문 내용:\n", target_element.get_text(strip=True))
else:
    print("요소를 찾지 못했습니다.")

자동으로 찾은 CSS Selector: html
본문 내용:
 “포르쉐도 흔해”···애스턴마틴·마세라티 등 최상위 틈새시장 공략 < 자동차 < 기업 < 기사본문 - 시사저널e주요서비스 바로가기본문 바로가기매체정보 바로가기로그인 바로가기기사검색 바로가기전체서비스 바로가기로그인회원가입facebookinstagramyoutube전체메뉴전체기사경제정책정책거시경제기업재계IT자동차철강화학유통항공서비스조선중소기업통신스타트업엔터테인먼트부동산부동산시황건설부동산정책금융은행보험카드증권시황종목펀드채권국제국제경제미국중국일본아시아기타정치국회한반도제약/바이오테크인공지능블록체인사회생활/문화IR전체기사경제정책정책거시경제기업재계IT자동차철강화학유통항공서비스조선중소기업통신스타트업엔터테인먼트부동산부동산시황건설부동산정책금융은행보험카드금융정책증권시황종목펀드채권국제국제경제미국중국일본아시아기타정치국회한반도제약/바이오테크인공지능블록체인사회쓰다창업기피플생활/문화칼럼데스크칼럼기자수첩오피니언영상기사머니방위대시승센터현장&이슈세상을it는리뷰e.Shorts카드뉴스IRADVERTORIALPEOPLE사고인사부음검색기사검색검색HOME기업자동차자동차“포르쉐도 흔해”···애스턴마틴·마세라티 등 최상위 틈새시장 공략박성수기자입력 2024.10.23 10:11댓글 0북마크본문 글씨 키우기본문 글씨 줄이기본문크기본문 글씨 줄이기본문 글씨 키우기북마크공유하기프린트이 기사를 공유합니다페이스북(으)로 기사보내기트위터(으)로 기사보내기카카오톡(으)로 기사보내기네이버밴드(으)로 기사보내기네이버블로그(으)로 기사보내기네이버라인(으)로 기사보내기핀터레스트(으)로 기사보내기텔레그램(으)로 기사보내기URL복사(으)로 기사보내기닫기애스턴마틴·마세라티, 올해 한국 시장 진출 본격화···법인 출범 및 공식 수입사 선정고가 수입차 시장 성장 속 빈틈 찾기 전략···작년 억대 수입차 비중 30% 육박연두색 번호판 적용 따른 고가 법인차 판매 감소 추이는 변수[시사저널e=박성수 기자] 올해 애스턴마틴, 마세라티 등 슈퍼카 브랜드들이 한국 시장 판매 확대를

In [14]:
def get_unique_selector(element):
    path = []
    while element.parent:
        siblings = element.parent.find_all(element.name, recursive=False)
        if len(siblings) > 1:
            index = siblings.index(element) + 1
            path.append(f'{element.name}:nth-of-type({index})')
        else:
            path.append(element.name)
        element = element.parent
    return ' > '.join(reversed(path))

url = 'https://www.sisajournal-e.com/news/articleView.html?idxno=406663'

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, 'lxml')

target_text_snippet = '올해'

target_element = soup.find(lambda tag: tag.name and tag.get_text() and target_text_snippet in tag.get_text())

if target_element:
    selector = get_unique_selector(target_element)
    print("자동으로 찾은 CSS Selector:", selector)

    content_tag = soup.select_one(selector)
    content = content_tag.get_text(separator='\n', strip=True)

    print("\n본문 내용:\n", content)
else:
    print("본문 요소를 찾지 못했습니다.")

자동으로 찾은 CSS Selector: html

본문 내용:
 “포르쉐도 흔해”···애스턴마틴·마세라티 등 최상위 틈새시장 공략 < 자동차 < 기업 < 기사본문 - 시사저널e
주요서비스 바로가기
본문 바로가기
매체정보 바로가기
로그인 바로가기
기사검색 바로가기
전체서비스 바로가기
로그인
회원가입
facebook
instagram
youtube
전체메뉴
전체기사
경제정책
정책
거시경제
기업
재계
IT
자동차
철강
화학
유통
항공서비스
조선
중소기업
통신
스타트업
엔터테인먼트
부동산
부동산시황
건설
부동산정책
금융
은행
보험
카드
증권
시황
종목
펀드
채권
국제
국제경제
미국
중국
일본
아시아
기타
정치
국회
한반도
제약/바이오
테크
인공지능
블록체인
사회
생활/문화
IR
전체기사
경제정책
정책
거시경제
기업
재계
IT
자동차
철강
화학
유통
항공서비스
조선
중소기업
통신
스타트업
엔터테인먼트
부동산
부동산시황
건설
부동산정책
금융
은행
보험
카드
금융정책
증권
시황
종목
펀드
채권
국제
국제경제
미국
중국
일본
아시아
기타
정치
국회
한반도
제약/바이오
테크
인공지능
블록체인
사회
쓰다창업기
피플
생활/문화
칼럼
데스크칼럼
기자수첩
오피니언
영상기사
머니방위대
시승센터
현장&이슈
세상을it는리뷰
e.Shorts
카드뉴스
IR
ADVERTORIAL
PEOPLE
사고
인사
부음
검색
기사검색
검색
HOME
기업
자동차
자동차
“포르쉐도 흔해”···애스턴마틴·마세라티 등 최상위 틈새시장 공략
박성수
기자
입력 2024.10.23 10:11
댓글 0
북마크
본문 글씨 키우기
본문 글씨 줄이기
본문크기
본문 글씨 줄이기
본문 글씨 키우기
북마크
공유하기
프린트
이 기사를 공유합니다
페이스북(으)로 기사보내기
트위터(으)로 기사보내기
카카오톡(으)로 기사보내기
네이버밴드(으)로 기사보내기
네이버블로그(으)로 기사보내기
네이버라인(으)로 기사보내기
핀터레스트(으)로 기사보내기
텔레그램(으)로 기사보내기
URL복사(으)로 기사보내기
닫기
애스턴마틴·마세라티, 올해 한국 시장

In [9]:
import requests
from bs4 import BeautifulSoup
import csv

# 대상 URL
url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=연두색+번호판"

# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

# 페이지 요청
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

# FAQ 리스트 추출 (일반적으로는 class가 'faq_question' 등으로 구성됨)
faq_items = soup.select('.faq_question')  # 질문
faq_answers = soup.select('.faq_answer')  # 답변

# 추출한 데이터를 리스트에 저장
faq_data = []
for question, answer in zip(faq_items, faq_answers):
    q_text = question.get_text(strip=True)
    a_text = answer.get_text(strip=True)
    faq_data.append([q_text, a_text])

# CSV로 저장
with open('naver_faq.csv', 'w', newline='', encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    writer.writerow(['Question', 'Answer'])
    writer.writerows(faq_data)

print("크롤링 완료! naver_faq.csv 파일을 확인하세요.")


크롤링 완료! naver_faq.csv 파일을 확인하세요.
